In [1]:
# is a good pratice only import what you need in the notebook , ill refactor this later in these notebooks :)

import sys
sys.path.append(r'C:\Users\bield\Documents\projsData\proj-1\utils')
from utils import (createSparkSesion,
                  takePandas)
from pyspark.sql import (functions as F,
                        Window)
import matplotlib.pyplot as plt



In [22]:
sys.path.append(r'C:\Users\bield\Documents\projsData\proj-1\utils')

In [8]:
spark = createSparkSesion()

In [ ]:
prefix = r"../../datasets/download/Atividades/"


In [30]:
df_clients     = spark.read.parquet(f"{prefix}/Clientes.parquet")
df_sales_itens = spark.read.parquet(f"{prefix}/ItensVendas.parquet")
df_products    = spark.read.parquet(f"{prefix}/Produtos.parquet")
df_sales       = spark.read.parquet(f"{prefix}/Vendas.parquet")
df_seller      = spark.read.parquet(f"{prefix}/Vendedores.parquet")

In [31]:
df_clients.takePandas(2)

,ClienteID,Cliente,Estado,Genero,Status
0,1,Adelina Buenaventura,RJ,M,Silver
1,2,Adelino Gago,RJ,M,Silver


In [33]:
df_sales_itens.takePandas(2)

,ProdutoID,VendasID,Quantidade,ValorUnitario,ValorTotal,Desconto,TotalComDesconto
0,2,400,2,9201.0,18402.0,"6256,68",12145.32
1,2,385,2,9201.0,18402.0,"5704,62",12697.38


In [34]:

df_products.takePandas(2)

,ProdutoID,Produto,Preco
0,1,Bicicleta Aro 29 Mountain Bike Endorphine 6.3 ...,"8.852,00"
1,2,Bicicleta Altools Stroll Aro 26 Freio À Disco ...,"9.201,00"


In [35]:
df_sales.takePandas(2)

,VendasID,VendedorID,ClienteID,Data,Total
0,1,1,91,1/1/2019,8053.6
1,2,6,185,1/1/2020,150.4


In [36]:
df_seller.takePandas(2)

,VendedorID,Vendedor
0,1,Armando Lago
1,2,Capitolino Bahía


In [ ]:
#my objective here is not to do a profund business analysis , just a few insights to build a pyspark code , and exemplify my abilities in build a ETL , desnormazile tables etc...

In [ ]:
#to do

#sales mans metrics
#with seller give more discounts
#best sellers by month >by amount , by sales
#objective  > identify best emplooyes ,for maybe a bonus , and find a possible corelation between give discounts, and be the best seller



#clients metrics 
#top client , by amount , by recurrence
#maybe detect with client we could offer discounts , send marketing campaings, trigger some fidelity bonus etc...




#store metrics:
#month over month
#month distribuition , by amount , by untary sales
#understando general store metrics



#product metrics:
#top product , by unitys sold, by amount
#understand which product we sell more , for some kind of storage managment , tendencies etc..



<h1>Sellers metrics 

a maneger could use this etl to track sellers performance , for example

In [112]:
df_sellers_metrics  = (
                        df_sales.join(df_seller,on="VendedorID",how="left")
                                .join(df_sales_itens,on="VendasID",how="left")
                                .select(
                                        F.expr("VendasID as sale_id"),
                                        F.expr("Vendedor as seller_name"),
                                        F.expr("Total as total_sale_amount"),
                                        F.expr("ProdutoId as product_id"),
                                        F.expr("TotalComDesconto as item_final_price_with_discont"),
                                        F.expr("Desconto as discont"),
                                        F.expr("Data as date")
                                        )
                                .withColumn("discont",
                                                        F.when(F.col("discont").rlike(r"-"),0.00)
                                                         .otherwise(
                                                                        F.regexp_replace(F.col("discont"),r",",r".")
                                                                        .cast("double")) # tratando os nulos
                                            )
                                .withColumn("date",F.to_date(F.to_timestamp(F.col("date"), "d/M/yyyy"))) # cast para time stamp facilita funcoes de calculo e operacoes com datas
                                .withColumn("month",F.month(F.col("date")))
                                .withColumn("year",F.year(F.col("date")))
                                .drop("date")
                        )

In [130]:
total_sales_value_window        = Window.partitionBy("month","year").orderBy(F.desc("total_sales_value"))
total_sales_window              = Window.partitionBy("month","year").orderBy(F.desc("total_sales"))
total_discount_given_window     = Window.partitionBy("month","year").orderBy(F.desc("total_discount_given"))

In [131]:
top_sellers = (df_sellers_metrics
                                        .groupBy("month","year","seller_name")
                                        .agg(
                                            F.sum("total_sale_amount").alias("total_sales_value"),
                                            F.count("sale_id").alias("total_sales"),
                                            F.sum("discont").alias("total_discount_given")
                                            )
                                        .withColumn("top_by_amount",F.rank().over(total_sales_value_window))
                                        .withColumn("top_by_sales",F.rank().over(total_sales_window))
                                        .withColumn("top_by_discont",F.rank().over(total_discount_given_window))
                                        .select("month","year","seller_name","total_sales_value","top_by_amount","total_sales","top_by_sales","total_discount_given","top_by_discont")#just to order column orders
                        )

In [132]:
top_sellers.filter((F.col("month")==1)&(F.col("year")==2020)).takePandas(20)

,month,year,seller_name,total_sales_value,top_by_amount,total_sales,top_by_sales,total_discount_given,top_by_discont
0,1,2020,Hélio Liberato,18402.00,5,1,7,184.02,1
1,1,2020,Godo Capiperibe,47467.20,1,3,4,88.52,2
2,1,2020,Jéssica Castelão,37258.58,3,11,1,78.28,3
3,1,2020,Daniel Pirajá,40878.12,2,4,2,61.54,4
4,1,2020,Iberê Lacerda,9711.88,6,3,4,24.19,5
5,1,2020,Armando Lago,526.16,7,2,6,2.63,6
6,1,2020,Capitolino Bahía,33357.08,4,4,2,0.00,7


In [ ]:
# after all this ET(L) process i could Load this date somewhere , maybe in a db where a manager could access  , schedulle this pipe on airflow etc..

<h1>clients metrics